**Provenance du code :** https://github.com/pmongruel/StatApp/blob/main/Data%20Process/importation_donnees.ipynb 

This Notebook :
- import data from French National Assembly website
- create 3 dataset : 'database_votants_herve.csv', 'database_scrutins_herve.csv' and 'députés.csv'

In [2]:
import zipfile
import io
import urllib.request
import json
import pandas as pd

access_url = urllib.request.urlopen('http://data.assemblee-nationale.fr/static/openData/repository/15/loi/scrutins/Scrutins_XV.json.zip')

data_dict=dict()

z = zipfile.ZipFile(io.BytesIO(access_url.read()))
n_scrutin = len(z.infolist())


for i in range(n_scrutin):
  data = json.loads(z.read(z.infolist()[i]).decode())
  data_dict[i] = data

In [10]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [12]:
from tqdm.notebook import tqdm

df_votants = pd.DataFrame(columns=['idScrutin', 'idVotant', 'vote', 'organeRefGroupe'])

for idScrutin, scrutin in tqdm(data_dict.items()):
    example = scrutin['scrutin']
    ventilation_dict = example['ventilationVotes']['organe']

    organeRefTot= ventilation_dict['organeRef']
    groupe_lists = ventilation_dict['groupes']['groupe']

    for groupe_dict in groupe_lists:
        organe_ref_groupe = groupe_dict['organeRef']
        n_groupe = groupe_dict['nombreMembresGroupe']
        vote_dict = groupe_dict['vote']

        decompteVoix_groupe_dict = vote_dict['decompteVoix']
        decompteNominatif_groupe_dict = vote_dict['decompteNominatif']

        liste_non_votant_groupe = decompteNominatif_groupe_dict['nonVotants']
        liste_pours_groupe = decompteNominatif_groupe_dict['pours']
        liste_contres_groupe = decompteNominatif_groupe_dict['contres']

        if not liste_non_votant_groupe is None:
            if type(liste_non_votant_groupe['votant']) == dict:
                df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant': liste_non_votant_groupe['votant']['acteurRef'], 'vote' : 'Pour', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)
            else :
                for vote in liste_non_votant_groupe['votant']:
                    df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant' : vote['acteurRef'], 'vote' : 'Non-votant', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)

        if not liste_pours_groupe is None :
            if type(liste_pours_groupe['votant']) == dict:
                df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant':liste_pours_groupe['votant']['acteurRef'], 'vote' : 'Pour', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)
            else :
                for vote in liste_pours_groupe['votant']:
                    df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant' : vote['acteurRef'], 'vote' : 'Pour', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)

        if not liste_contres_groupe is None:
            if type(liste_contres_groupe['votant']) == dict:
                df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant':liste_contres_groupe['votant']['acteurRef'], 'vote' : 'Contre', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)
            else :
                for vote in liste_contres_groupe['votant']:
                    df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant' : vote['acteurRef'], 'vote' : 'Contre', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)

df_votants

,idScrutin,idVotant,vote,organeRefGroupe
0,0,PA606171,Non-votant,PO730964
1,0,PA721824,Non-votant,PO730964
2,0,PA719952,Pour,PO730964
3,0,PA607395,Pour,PO730964
4,0,PA718710,Pour,PO730964
...,...,...,...,...
450096,4416,PA335612,Pour,PO730940
450097,4416,PA607619,Contre,PO759900
450098,4416,PA719608,Pour,PO723569
450099,4416,PA721932,Pour,PO723569


In [13]:
df_votants.to_csv('database_votants_herve.csv')

In [14]:
df_scrutin = pd.DataFrame(columns = ['idScrutin', 'date_scrutin', 'code_type_vote', 'titre', 'demandeur', 'organe_ref', 'votants', 
                                     'pour', 'contre', 'non_votants', 'non_votants_volontaires', 'abstention', 'resultat'])

for idScrutin, scrutin in data_dict.items():
    scrutin = scrutin['scrutin']
    df_scrutin = df_scrutin.append({'idScrutin' : idScrutin, 'date_scrutin' : scrutin['dateScrutin'], 'code_type_vote': scrutin['typeVote']['codeTypeVote'],
                  'titre' : scrutin['titre'], 'demandeur' : scrutin['demandeur']['texte'], 'organe_ref' : scrutin['organeRef'],
                  'votants': scrutin['syntheseVote']['nombreVotants'], 'pour' : scrutin['syntheseVote']['decompte']['pour'], 
                  'contre' : scrutin['syntheseVote']['decompte']['contre'], 'non_votants' : scrutin['syntheseVote']['decompte']['nonVotants'],
                  'non_votants_volontaires' : scrutin['syntheseVote']['decompte']['nonVotantsVolontaires'],
                  'abstention': scrutin['syntheseVote']['decompte']['abstentions'],'resultat': scrutin['sort']['code']}, ignore_index=True)

df_scrutin

,idScrutin,date_scrutin,code_type_vote,titre,demandeur,organe_ref,votants,pour,contre,non_votants,non_votants_volontaires,abstention,resultat
0,0,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57,56,0,2,0,1,adopté
1,1,2021-02-12,SPO,l'article 33 du projet de loi confortant le re...,"Président du groupe ""La République en Marche""",PO717460,97,83,3,2,0,11,adopté
2,2,2019-03-15,SPO,l'amendement n°23 de M. Potier à l'article 62 ...,"Présidente du groupe ""Socialistes et apparentés""",PO717460,35,9,26,2,0,0,rejeté
3,3,2019-11-13,SPO,"la motion de rejet préalable, déposée par Mme ...","Présidente du groupe ""Socialistes et apparentés""",PO717460,93,33,59,2,0,1,rejeté
4,4,2017-10-19,SPO,l'amendement n° 400 de Mme Louwagie à l'articl...,"Président du groupe ""Les Républicains""",PO717460,107,19,87,2,0,1,rejeté
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4412,4412,2020-10-30,SPO,l'amendement n° 1237 de la commission des fina...,"Présidente du groupe ""Socialistes et apparentés""",PO717460,73,26,41,2,0,6,rejeté
4413,4413,2021-04-16,SPO,le sous-amendement n° 7434 de M. Millienne et ...,"Président du groupe ""La France insoumise""",PO717460,121,21,93,2,0,7,rejeté
4414,4414,2020-07-31,SPO,l'amendement n° 1074 de M. Bazin et les amende...,"Président du groupe ""Les Républicains""",PO717460,74,17,55,2,0,2,rejeté
4415,4415,2021-07-22,SPO,l'amendement n° 92 de M. Pauget et les amendem...,"Président du groupe ""Les Républicains""",PO717460,165,76,82,1,0,7,rejeté


In [15]:
df_scrutin.to_csv('database_scrutins_herve.csv')

In [11]:
url = 'https://data.assemblee-nationale.fr/static/openData/repository/15/amo/deputes_actifs_csv_opendata/liste_deputes_libre_office.csv'

df_deputes = pd.read_csv(url)
df_deputes.to_csv('députés.csv')